# Contents
1. open csv, merge, check and remove duplicates
2. ensure the right chats were scraped
3. check for and remove duplicates and NA in text column
4. remove chats that are not part of Querdenken community (mainstream media)
5. apply classification of party, media, movement
6. create message_id
7. check user/chats relation per category of chat, remove duplicate channels due to case sensitivity
8. investigate hashtags
9. check datetime, create cutoff point (1 Jan 2020)
10. check views
11. preparing edgelist for network analysis
12. dropping unnecessary columns, create 'text_clean'
13. clean text with script (URL, lowercase, special characters)
14. create subset and remove barely seen messages
15. remove very short messages
16. investigate short messages and grouping possibilities
17. save new df_textanal_short to csv

**end**

### step 1: Open csv, merge, check and remove duplicates
from the three iterations of scraping (s1,s2,s3), the output csv are merged and checked for duplicate rows

In [ ]:
import pandas as pd

# Paths to your CSV files
csv_file_paths = ['output_s1.csv', 'output_s2.csv', 'output_s3_t5.csv']

# Read the CSV files into DataFrames
df1 = pd.read_csv(csv_file_paths[0])
df2 = pd.read_csv(csv_file_paths[1])
df3 = pd.read_csv(csv_file_paths[2])

# 1. Check if all column names of the three CSVs are identical
columns_identical = (df1.columns == df2.columns).all() and (df2.columns == df3.columns).all()

# 2. See how many rows each CSV has
rows_df1 = len(df1)
rows_df2 = len(df2)
rows_df3 = len(df3)

# 3. Merge the three df into a new one called 'merged_df'
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

# 4. Check if there are rows that are identical (duplicates) in the new csv
duplicates_in_merged = merged_df.duplicated().any()

# Output the results
print(f"Are all column names identical across the three CSVs? {columns_identical}")
print(f"Number of rows in CSV 1: {rows_df1}")
print(f"Number of rows in CSV 2: {rows_df2}")
print(f"Number of rows in CSV 3: {rows_df3}")
print(f"Are there any duplicate rows in 'output_final_t5.csv'? {duplicates_in_merged}")

In [ ]:
number_of_duplicates = merged_df.duplicated(keep='first').sum()
print(f"Number of duplicate rows: {number_of_duplicates}")
merged_df = merged_df.drop_duplicates()
print("duplicate rows have been removed")

Now check for duplicates in 'link_to_message' column

In [ ]:
duplicates = merged_df.duplicated(subset='link_to_message', keep=False)

# Counting the number of duplicates
number_of_duplicates = duplicates.sum()

# Printing the number of duplicates
print(f"Number of duplicate entries in 'link_to_message': {number_of_duplicates}")

In [ ]:
merged_df = merged_df.drop_duplicates(subset= "link_to_message", keep='first')
duplicates = merged_df.duplicated(subset='link_to_message', keep=False)

# Counting the number of duplicates
number_of_duplicates = duplicates.sum()

# Printing the number of duplicates
print(f"Number of duplicate entries in 'link_to_message': {number_of_duplicates}")

now the dataframe merged_df does not contain duplicate messages. The text of some messages may still be identical, but messages that were scraped multiple times (in the different seeds) have been removed (through removing duplicates of the unique links to the messages).

### step 2: check that only chats from the seeds were scraped
qualitative assessment of those that are not from seeds: Retain y/n

In [ ]:
print(merged_df.columns.tolist())

In [ ]:
# Load channel names from the first .txt file
with open('/home/onyxia/work/new/seeds/seed2.txt', 'r') as file:
    channels_list_1 = [line.strip() for line in file]

# Load channel names from the second .txt file
with open('/home/onyxia/work/new/seeds/seed3_t5.txt', 'r') as file:
    channels_list_2 = [line.strip() for line in file]
combined_channel_list = list(set(channels_list_1 + channels_list_2))
unmatched_channels = merged_df[~merged_df['channel_name'].isin(combined_channel_list)]
print("Unmatched channel names:")
print(unmatched_channels['channel_name'].unique())

Qualitative Assessment of the Chats: belong the movement/party. So they will be kept.

### step 3: check for and remove duplicates and NA in text column

In [ ]:
#1 remove na in text column
# Count rows where 'text' is NA
na_count = merged_df['text'].isna().sum()
print(f"Number of rows with NA in 'text' column: {na_count}")

# Remove rows where 'text' is NA
df_clean = merged_df.dropna(subset=['text']).copy()

# Verify removal
print(f"Number of rows after removing NAs: {len(df_clean)}")

### step 4: remove chats from mainstream media

In [ ]:
#3. remove what is not part of Querdenken - check for keywords of mainstream media or main parties in 'channel_name'
media_keywords = [
    "ARD", "ZDF", "Deutschlandfunk", "NDR", "WDR", 
    "SWR", "BR", "MDR", "rbb", "DW", 
    "Die Zeit", "FAZ", "Frankfurter Allgemeine", "Süddeutsche Zeitung", 
    "Der Spiegel", "Stern", "Focus", "Bild", "SPD", "FDP", "CDU", "AFD", "Linke", "Grüne"
]

media_keywords_lower = [keyword.lower() for keyword in media_keywords]

def contains_media_keyword(channel_name):
    channel_name_lower = channel_name.lower() if isinstance(channel_name, str) else ""
    return any(keyword in channel_name_lower for keyword in media_keywords_lower)
    
media_related_channel_names = [channel for channel in df_clean['channel_name'] if contains_media_keyword(channel)]

media_related_channel_names_unique = list(set(media_related_channel_names))

print("Unique channel names containing keywords related to German mainstream media:")
for channel in media_related_channel_names_unique:
    print(channel)

no mainstream media channels detected based on keywords

### step 5: apply classification of media/movement/party

In [ ]:
#categorise channels into party, movement, media
import pandas as pd

# Sample DataFrame creation (replace this with your actual df_clean DataFrame)
# df_clean = pd.DataFrame({
#     'channel_name': ['basisSomething', 'InfoMediaChannel', 'TVChannel', 'MixedBasisTV']
# })

# Initialize 'channel_category' with default value 0
df_clean['channel_category'] = 0

# Function to categorize channel based on 'channel_name'
def categorize_channel(channel_name):
    channel_name_lower = channel_name.lower()
    if 'basis' in channel_name_lower:
        return 'party'
    elif any(keyword in channel_name_lower for keyword in ['tv', 'info', 'media', 'news', 'magazin']):
        return 'media'
    return 'movement'  # Return 'movement'

# Apply the categorization function to the 'channel_name' column
df_clean['channel_category'] = df_clean['channel_name'].apply(categorize_channel)

# Display the DataFrame to verify the new 'channel_category' column
print(df_clean)

### step 6: add message_id

In [ ]:
#create id for each message
df_clean['message_id'] = df_clean.index

### step 7: check user/chats relation per category of chat

In [ ]:
#5 user column
unique_users = df_clean['user'].nunique()
print(f"Number of unique users: {unique_users}")

# Count how many messages each user sent
messages_per_user = df_clean['user'].value_counts()

print("Messages sent per user (top users):")
print(messages_per_user.head())

In [ ]:
matches = df_clean[df_clean['channel_name'] == df_clean['user']]
matches_count = len(matches)
print(f"Number of cases where 'channel_name' equals 'user': {matches_count}")
matches = df_clean[df_clean['channel_name'] == df_clean['user']]
matches_count_per_category = matches.groupby('channel_category').size()
print(matches_count_per_category)

In [ ]:
partei_users = set(df_clean[df_clean['channel_category'] == 'Partei']['user'].unique())
media_users = set(df_clean[df_clean['channel_category'] == 'Media']['user'].unique())
other_users = set(df_clean[df_clean['channel_category'] == '0']['user'].unique())
# Identify overlaps between each pair of categories
overlap_partei_media = partei_users.intersection(media_users)
overlap_partei_other = partei_users.intersection(other_users)
overlap_media_other = media_users.intersection(other_users)

# Identify users present in all three categories
overlap_all_three = partei_users.intersection(media_users, other_users)
print(f"Overlap between three categories {overlap_all_three}")
print(f"Overlap between Partei and media {overlap_partei_media}")
print(f"Overlap between Partei and Movement {overlap_partei_other}")
print(f"Overlap between Media and Movement {overlap_media_other}")

In [ ]:
#clean how many channels have been mentioned multiple times due to different capitalisations
df_clean['channel_name_lower'] = df_clean['channel_name'].str.lower()

# Count the unique, case-insensitive 'channel_name' occurrences
unique_channels_case_insensitive = df_clean['channel_name_lower'].nunique()

# Count the original, case-sensitive 'channel_name' occurrences
unique_channels_case_sensitive = df_clean['channel_name'].nunique()

# Calculate the difference to see how many are affected by case sensitivity
difference = unique_channels_case_sensitive - unique_channels_case_insensitive

print(f"Unique channels (case-sensitive): {unique_channels_case_sensitive}")
print(f"Unique channels (case-insensitive): {unique_channels_case_insensitive}")
print(f"Number of channels affected by case sensitivity: {difference}")
# Group by the lowercase channel name and count the unique case-sensitive names for each
channels_with_multiple_capitalizations = df_clean.groupby('channel_name_lower')['channel_name'].nunique()

# Filter for groups with more than one unique case-sensitive name
channels_with_multiple_capitalizations = channels_with_multiple_capitalizations[channels_with_multiple_capitalizations > 1]

print(f"Channels with multiple capitalizations: {len(channels_with_multiple_capitalizations)}")
print(channels_with_multiple_capitalizations)

In [ ]:
# Make 'channel_name' and 'user' lowercase
df_clean['channel_name_lower'] = df_clean['channel_name'].str.lower()
df_clean['user_lower'] = df_clean['user'].str.lower()

# Count unique values in the original and lowercase columns for 'user'
unique_users_case_sensitive = df_clean['user'].nunique()
unique_users_case_insensitive = df_clean['user_lower'].nunique()

# Calculate the difference to see how many are affected by case sensitivity for 'user'
difference_users = unique_users_case_sensitive - unique_users_case_insensitive

print(f"Unique users (case-sensitive): {unique_users_case_sensitive}")
print(f"Unique users (case-insensitive): {unique_users_case_insensitive}")
print(f"Number of users affected by case sensitivity: {difference_users}")

# Drop the original 'channel_name' and 'user' columns
df_clean = df_clean.drop(['channel_name', 'user'], axis=1)

# Rename the lowercase columns to the original column names
df_clean = df_clean.rename(columns={'channel_name_lower': 'channel_name', 'user_lower': 'user'})


In [ ]:
print(df_clean.columns.tolist())

### step 8: investigate hashtags

In [ ]:
#check hashtags
temp_df = (df_clean.dropna(subset=['hashtags'])
           .assign(individual_hashtags=lambda x: x['hashtags'].str.split(','))
           .explode('individual_hashtags'))

# Clean up individual hashtags: strip whitespace and convert to lowercase
temp_df['individual_hashtags'] = temp_df['individual_hashtags'].str.strip().str.lower()

# Group by 'channel_category' and 'individual_hashtags' and count frequencies
hashtag_counts = (temp_df.groupby(['channel_category', 'individual_hashtags'])
                  .size()
                  .reset_index(name='count'))

# Step 3: Sort within each 'channel_category' to identify the most frequent hashtags
sorted_hashtag_counts = hashtag_counts.sort_values(by=['channel_category', 'count'], ascending=[True, False])

# display top N hashtags for each category
top_n = 5
top_hashtags_by_category = sorted_hashtag_counts.groupby('channel_category').head(top_n)

print(top_hashtags_by_category)

### step 9: check time window and generate cutoff point (1 Jan 2020)

In [ ]:
#check time window
df_clean['datetime'] = pd.to_datetime(df_clean['datetime'])

min_datetime = df_clean['datetime'].min()
max_datetime = df_clean['datetime'].max()

print(f"Earliest datetime in the dataset: {min_datetime}")
print(f"Latest datetime in the dataset: {max_datetime}")

In [ ]:
cutoff_datetime = pd.Timestamp('2020-03-01', tz='UTC')

# Now you can safely compare and count messages sent before March 2020
messages_before_march_2020 = df_clean[df_clean['datetime'] < cutoff_datetime].shape[0]

print(f"Number of messages sent before March 2020: {messages_before_march_2020}")


In [ ]:
cutoff_datetime_jan_2020 = pd.Timestamp('2020-01-01', tz='UTC')

# Keep only messages on or after January 2020
df_clean = df_clean[df_clean['datetime'] >= cutoff_datetime_jan_2020]

In [ ]:
#check all went well
min_datetime = df_clean['datetime'].min()
max_datetime = df_clean['datetime'].max()

print(f"Earliest datetime in the dataset: {min_datetime}")
print(f"Latest datetime in the dataset: {max_datetime}")

### step 10: inspect views and create thresholds

In [ ]:
# Updated thresholds and labels to match the desired ranges
thresholds = [100000, 10000, 1000, 100, 10]
threshold_labels = ['100000>', '10000-99999', '1000-9999', '100-999', '10-99']

# Updated function to categorize views into thresholds
def categorize_views(views):
    if views > 100000:
        return '100000>'
    elif views > 10000:
        return '10000-99999'
    elif views > 1000:
        return '1000-9999'
    elif views > 100:
        return '100-999'
    elif views > 10:
        return '10-99'
    else:
        return '0-10'

# Apply the updated categorization function to the 'views' column
df_clean['view_threshold'] = df_clean['views'].apply(categorize_views)

### step 11: create edgelist for network analysis; 
both edgelist(containing all connections even to chats outside the community, and edgelist_clean which replaces chats not in the community with 'other_chats'

In [ ]:
print(df_clean.columns.tolist())

In [ ]:
# gephi df_network as csv with the cleaned datafile
import re
df_network = df_clean[['channel_name', 'links', 'channel_category']].copy()
def extract_valid_telegram_username(link_text):
    # Ensure input is a string
    if not isinstance(link_text, str):
        return None
    # Define the pattern for extracting Telegram usernames
    telegram_pattern = re.compile(r'(?:https?://)?t\.me/([\w\d_-]+)')
    # Search for all occurrences of the pattern
    matches = telegram_pattern.findall(link_text)
    # Filter out any non-Telegram links or invalid entries
    valid_usernames = [match for match in matches if not any(ext in match for ext in ['http', 'https', '|'])]
    # Return the first valid Telegram username, if available
    return valid_usernames[0] if valid_usernames else None

# Apply the refined function to extract valid Telegram usernames
df_network['links'] = df_network['links'].apply(extract_valid_telegram_username)
df_network['links'] = df_network['links'].str.lower()

# Drop rows without a valid Telegram username
df_network = df_network.dropna(subset=['links'])
df_network = df_network.dropna(subset=['channel_name'])
df_network = df_network.dropna(subset=['channel_category'])

contains_floats = df_network['links'].apply(lambda x: isinstance(x, float)).any()

if contains_floats:
    print("The column contains float values.")
    df_network = df_network[~df_network['links'].apply(lambda x: isinstance(x, float))]
else:
    print("No float values in 'links'.")

# create edgelist for copy
edgelist = df_network.copy()
edgelist['source'] = edgelist['channel_name'].str.lower()
edgelist['target'] = edgelist['links'].str.lower()
edgelist['channel_category'] = edgelist['channel_category'].str.lower()

edgelist = edgelist.drop('channel_name', axis=1)
edgelist = edgelist.drop('links', axis=1)
# Display the first few rows of edgelist to verify
print(edgelist.head())
len(edgelist)


In [ ]:
import pandas as pd

# Step 1: Read the text file, convert to lowercase, and split into a list
with open('/path/to/seed3_t5.txt', 'r') as file:
    valid_targets = file.read().lower().splitlines()

# Step 2: Copy edgelist to edgelist_clean and convert 'target' in edgelist_clean to lowercase for comparison
edgelist_clean = edgelist.copy()
edgelist_clean['target'] = edgelist_clean['target'].str.lower()

# Step 3: Replace values in 'target' that are not in valid_targets with 'other_chats'
edgelist_clean['target'] = edgelist_clean['target'].apply(lambda x: x if x in valid_targets else 'other_chats')

# Note: Convert 'target' back to its original case if needed here

# Step 4: Your edgelist_clean is ready

In [ ]:
edgelist_clean.to_csv("path/to/edgelist_clean_v2.csv", index=False)
edgelist.to_csv("path/to/edgelist_v2.csv", index=False)

edgelist has been created. NA will continue in gephi

### step 12: drop unnecessary columns, create 'text_clean'

In [ ]:
print(df_clean.columns.tolist())

In [ ]:
# Columns to be dropped
columns_to_drop = [
    'user_img', 'nb_hashtags', 'hashtags', 
    'is_reply_img', 'is_reply_user', 'is_reply_text', 
    'stickers', 'nb_photos', 'photos', 
    'nb_videos', 'videos', 'videos_times', 
    'link_img', 'link_site', 'link_title', 'link_description', 
    'edited'
]
# Dropping the specified columns from df_clean
df_clean = df_clean.drop(columns=columns_to_drop)

# Displaying the first few rows to verify the columns have been dropped
print(df_clean.head())

In [ ]:
print(df_clean.columns.tolist())

In [ ]:
df_clean['text_clean'] = df_clean['text']
print(df_clean.columns.tolist())

In [ ]:
new_order = [
    'message_id', 'channel_name', 'channel_category', 
    'text', 'text_clean', 'views', 'view_threshold', 
    'datetime'
]

# Adding the remaining columns to the new order, ensuring no column is dropped
remaining_columns = [col for col in df_clean.columns if col not in new_order]
new_order += remaining_columns

# Reassigning the columns of df_clean based on the new order
df_clean = df_clean[new_order]


In [ ]:
print(df_clean.columns.tolist())

### step 13: clean text with script

the script removes "..." multiple periods, converts to lowercase, removes URLs

In [ ]:
import sys
sys.path.append('path/to/scripts')
from text_broom import text_broom
from text_broom_chill import text_broom_chill
# Apply text_broom_chill to text_notsoclean
df_clean['text_clean'] = df_clean['text_clean'].apply(text_broom_chill)

### step 14: create subset for further + remove barely seen messages

In [ ]:
# save old cleaned datafile 'df_clean' for future reference
df_clean.to_csv("path/to/big_csv_20March.csv", index=False)

In [ ]:
#new dataframe df_textanal for BERT
df_textanal = df_clean
df_textanal['channel_category'] = df_textanal['channel_category'].str.strip()

In [ ]:
unique_values_count = df_textanal['view_threshold'].value_counts()

# Find out how this distribution is different for the three categories in 'channel_category'
category_distribution = df_textanal.groupby('channel_category')['view_threshold'].value_counts().unstack(fill_value=0)

(unique_values_count, category_distribution)

In [ ]:
distribution_percent = category_distribution.div(category_distribution.sum(axis=1), axis=0) * 100

distribution_percent

For party chat messages(channel_category=party), ca. 50% were viewed by between 100 and 1000 people. another 30% were viewed by 10-100 people. Only 5% were viewed by more than 10.000 people.

For media chat messages, (channel_category=media), ca. 50% were seen by more than 10.000 people. 15% by >100 and another 15% by less than 100.

For Community chat messages, (channel_category=movement), 30% were seen by >10.000 people, 30% by more than 10.000 people, 25% by 100-10.000 people.

I would suggest dropping all messages <100 views from movement and media chats.

In [ ]:
# Keep rows that satisfy one of the following conditions:
# 1. The channel_category is 'party'
# 2. The channel_category is 'movement' or 'media', but views are 100 or more
df_textanal_short = df_textanal[
    (df_textanal['channel_category'] == 'party') |
    ((df_textanal['channel_category'].isin(['movement', 'media'])) & (df_textanal['views'] >= 100))
]

In [ ]:
# Calculate the number of rows in the original DataFrame
original_row_count = len(df_textanal)

# Calculate the number of rows in the filtered DataFrame
filtered_row_count = len(df_textanal_short)

# Calculate the number of rows dropped
rows_dropped = original_row_count - filtered_row_count

# Print the result
print(f"Number of rows dropped: {rows_dropped}")


### step 15: remove very short messages
check distribution of message length and then decide which messages to retain. Goal: greater uniformity for BERTopic

In [ ]:
contains_floats = df_textanal['text_clean'].apply(lambda x: isinstance(x, float)).any()

if contains_floats:
    print("The column contains float values.")
    df_textanal = df_textanal[~df_textanal['text_clean'].apply(lambda x: isinstance(x, float))]
else:
    print("No float values in 'text_clean'.")
contains_floats = df_textanal_short['text_clean'].apply(lambda x: isinstance(x, float)).any()

if contains_floats:
    print("The column contains float values.")
    df_textanal_short = df_textanal_short[~df_textanal['text_clean'].apply(lambda x: isinstance(x, float))]
else:
    print("No float values in 'text_clean'.")


In [ ]:
import pandas as pd

# Assuming df_textanal_short exists and has a column 'text_lowernopunct'
word_counts_per_cell = df_textanal_short['text_clean'].apply(lambda x: len(x.split()))

bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 130, 150, 170, 200, 250, float('inf')]
labels = ['<5', '6-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-110', '111-130', '131-150', '151-170', '171-200', '201-250', '251+']

word_count_categories = pd.cut(word_counts_per_cell, bins=bins, labels=labels, right=False)
word_count_distribution = word_count_categories.value_counts().sort_index()

print(word_count_distribution)


In [ ]:
import pandas as pd

# Define bins and labels
bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 130, 150, 170, 200, 250, float('inf')]
labels = ['<5', '6-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-110', '111-130', '131-150', '151-170', '171-200', '201-250', '251+']

categories = ["media", "movement", "party"]

results = {}

for category in categories:
    # Filter the DataFrame for the current channel_category
    df_subset = df_textanal_short[df_textanal_short['channel_category'] == category]
    
    # Calculate word count distribution for the subset
    word_counts_per_cell = df_subset['text_clean'].apply(lambda x: len(x.split()))
    word_count_categories = pd.cut(word_counts_per_cell, bins=bins, labels=labels, right=False)
    word_count_distribution = word_count_categories.value_counts().sort_index()
    
    # Store results
    results[category] = word_count_distribution

# Now, results contains the word count distributions for each channel_category subset
for category, distribution in results.items():
    print(f"Channel Category: {category}")
    print(distribution, "\n")


In [ ]:
#for better visualisation: 
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
for category, distribution in results.items():
    plt.plot(distribution.index, distribution.values, label=category)

# Add labels and legend
plt.title('Word Count Distribution by Channel Category')
plt.xlabel('Word Count Categories')
plt.ylabel('Frequency')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.tight_layout()  # Adjust layout to prevent overlap of labels
plt.show()

In [ ]:
df_textanal_short['word_count'] = df_textanal_short['text_clean'].apply(lambda x: len(x.split()))

# Filter the DataFrame for rows where the number of words is less than 10
df_less_than_10_words = df_textanal_short[df_textanal_short['word_count'] < 5]

In [ ]:
random_samples = df_less_than_10_words.sample(n=5)

pd.set_option('display.max_colwidth', None)
# Display the sampled rows
print(random_samples[['text_clean']])

may of the very short messages are currently na, which will be removed at a later stage.Those with less than 10 words are often quite meaningful. <5 words are not meaningful and can be removed right away from df_textanal_short.

In [ ]:
df_textanal_short = df_textanal_short[df_textanal_short['word_count'] >= 5]
print(len(df_textanal))
print(len(df_textanal_short))

### step 16: check if messages can be grouped together

In [ ]:
df_textanal_short['datetime'] = pd.to_datetime(df_textanal_short['datetime'])

# Sort the DataFrame by 'channel_name' and 'datetime'
df_textanal_short.sort_values(by=['channel_name', 'datetime'], inplace=True)
# Convert 'datetime' to just the date and time (without timezone) if not already
df_textanal_short['datetime'] = df_textanal_short['datetime'].dt.tz_localize(None)

# Initialize the 'thread' column with None or an appropriate default value
df_textanal_short['thread'] = None



In [ ]:
#now see average distance between messages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter the DataFrame for the channels of interest
channels_of_interest = ['die_basis_funkt', 'zwanzig4', 'haintz']
df_filtered = df_textanal_short[df_textanal_short['channel_name'].isin(channels_of_interest)]

# Ensure datetime is in the correct format and sorted
df_filtered['datetime'] = pd.to_datetime(df_filtered['datetime'])
df_filtered.sort_values(by=['channel_name', 'datetime'], inplace=True)

# Calculate time deltas within each channel
df_filtered['time_delta'] = df_filtered.groupby('channel_name')['datetime'].diff().dt.total_seconds() / 60  # delta in minutes

# Visualizing the distribution of time deltas
for channel in channels_of_interest:
    df_channel = df_filtered[df_filtered['channel_name'] == channel].dropna(subset=['time_delta'])
    
    # We use a logarithmic scale to better visualize the distribution, as it's likely to be heavily skewed
    plt.figure(figsize=(10, 6))
    plt.hist(df_channel['time_delta'], bins=np.logspace(np.log10(1), np.log10(df_channel['time_delta'].max()), 50), color='skyblue', edgecolor='black')
    plt.xscale('log')  # Set x-axis to logarithmic scale
    plt.title(f'Log-scaled Distribution of Time Deltas for {channel}')
    plt.xlabel('Time Delta (minutes, log scale)')
    plt.ylabel('Frequency')
    plt.grid(True, which="both", ls="--")
    plt.show()


**It appears that the messages are rather more spread out - only in the case of haintz (a news channel) is the difference smaller. Given the great diversity of chats, finding a one-size-fits-all approach would be impossible. Therefore grouping messages into threads is not possible.**

### step 17: save as a new csv

In [ ]:
df_textanal_short.to_csv("path/to/textanal_short.csv", index=False)